# CS 4496/7496 Computer Animation (Fall 2023)
Copyright (c) Georgia Institute of Technology


---


# Project 5: Twister (due 12/5/2023, Tuesday, 11:59PM)


![TWISER-IMAGE](https://upload.wikimedia.org/wikipedia/en/0/09/1966_Twister_Cover.jpg)

Have you played Twister? Your body is basically solving an inverse kinematics problem when you are playing Twister. Please refer to the video if you are not too familiar with it (https://www.youtube.com/watch?v=7A5XO0udmdo).

In this project, you will build a virtual Twister game by developing an IK solver.

Let us install the required libraries before we begin.
Similar to the previous assignments, we will be using PyBullet. (https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#)

In [ ]:
#@title Install dependencies
!apt-get install -y xvfb python-opengl ffmpeg
!pip install https://pybullet.org/download/pybullet-3.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install ffmpeg-python

# 1. Set up transformations.

In [ ]:
#@title Import required libraries
import pybullet as p
import numpy as np
from math import sin, cos
import scipy
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize

import os
import io
from matplotlib import pyplot as plt
import base64
import ffmpeg
from PIL import Image
from IPython.display import HTML
from IPython import display as ipythondisplay

# !wget "https://lh3.googleusercontent.com/drive-viewer/AK7aPaDMGbPI4WXfr9GBmMOGPiuTmOkomDPeXVKgrXzefHIiFpOQuXO8ZY1j3TG8aipbi067Q5WFqj4yn1WmS2SkJYGcQTeIww=s2560"


Next, we define the transformations that we will make use of. We will first define a base class, followed by the fixed (0 DOF) and translation (1 DOF) transformations that inherit from this base class.

For simplicity, we do not have a joint with multiple degrees of freedom. This is why our translation and rotation transformations only involve one axis (X/Y/Z) per object. We will instead use multiple objects of these types, one for each axis desired, to account for the degrees of freedom we need.

In [ ]:
#@title `Transformation` base class, subclasses, utilities

def transform_between(start, end):
  """This is a helper function to compute the transformation between start and
  end transformations.
  """
  if start == end:
    return np.identity(4)
  else:
    return transform_between(start, end.parent) @ end.local_transform()


class Transformation(object):
  """ Base class for all transformations that we subsequently define. """
  def __init__(self, name=None):
    global transform_list
    transform_list.append(self)
    self.name = name
    self.parent = None  # If parent is none, it is a root node.
    self.children = list()
    self.dependent_transformations = set()

  def add(self, child):
    child.parent = self
    self.children.append(child)
    return child

  def global_transform(self):
    return transform_between(None, self)

  def global_position(self):
    return self.global_transform()[:3, 3]

  def local_transform(self):
    raise NotImplementedError

  def local_derivative(self):
    raise NotImplementedError

  def num_dofs(self):
    raise NotImplementedError

  def get_dof_value(self):
    raise NotImplementedError

  def set_dof_value(self, value=0.0):
    raise NotImplementedError

  def __repr__(self):
    return "%s(%s)" % (self.__class__.__name__, self.name)


class Fixed(Transformation):
  """ Fixed transformation. """
  def __init__(self, name, tx=0.0, ty=0.0, tz=0.0, rx=0.0, ry=0.0, rz=0.0):
    super().__init__(name=name)
    self._value = np.identity(4)
    self._value[:3, :3] = R.from_euler('xyz', [rx, ry, rz]).as_matrix()
    self._value[:3, 3] = (tx, ty, tz)

  def num_dofs(self):
    return 0

  def local_transform(self):
    return self._value


class Translation(Transformation):
  """ Class for translation. """
  def __init__(self, name, axis, value=0.0):
    super().__init__(name=name)
    self._value = value
    self.axis = axis
    self.axis_index = "xyz".find(axis)
    self.T = np.identity(4)
    self.dT = np.zeros((4, 4))

  def num_dofs(self):
    return 1

  def get_dof_value(self):
    return self._value

  def set_dof_value(self, value=0.0):
    self._value = value

  def local_transform(self):
    self.T[self.axis_index, 3] = self._value
    return self.T

  def local_derivative(self):
    self.dT[self.axis_index, 3] = 1.0
    return self.dT

## 1.1 Hinge transformation.

 We have already defined the base class, the fixed transformation, and the translation transformation. Your task is to complete the rotation transformation.

 Please implement the `local_transform()` and `local_derivative()` methods as described in their docstrings.

 As per the slides, the derivative here is taken with respect to the single free variable of the transformation (as it has 1 DOF). So, for the hinge transformation's `local_derivative()` method, you should return the derivative with respect to the angle of rotation `_theta`.

 **<font color='orange'> \*\* Tasks 1 and 2: write your code below (10 + 10 = 20 pts) \*\* </font>**
   
**<font color='orange'> \*\* Note: The implementations of Task 1 and 2 are given. However, we recommend students to read the code carefully. \*\* </font>**

In [ ]:
class Hinge(Transformation):
  """Class for rotation. """
  def __init__(self, name, axis, theta=0.0):
    super().__init__(name=name)
    self._theta = theta
    self.axis = axis
    self.axis_index = "xyz".find(axis)

  def num_dofs(self):
    return 1

  def get_dof_value(self):
    return self._theta

  def set_dof_value(self, value=0.0):
    self._theta = value

  def local_transform(self):
    """ This function is used to compute the transformation matrix for a
    rotation operation. You are supposed to make use of the self._theta
    parameter and the sin and cos functions which have already been imported
    directly as `from math import sin, cos`.
    You will have 3 cases depending on which axis the rotation is about, which
    you can obtain from either self.axis ('x', 'y', z') or self.axis_index
    (0, 1, 2).

    Returns:
      transform: The 4x4 transformation matrix.
    """
    cth, sth = cos(self._theta), sin(self._theta)
    # Student code starts here. Implement three transformation matrices for each case.
    if self.axis == 'x':
      return np.array([[1.0, 0.0, 0.0, 0.0],
                       [0.0, cth, -sth, 0.0],
                       [0.0, sth, cth, 0.0],
                       [0.0, 0.0, 0.0, 1.0]])
    elif self.axis == 'y':
      return np.array([[cth, 0.0, sth, 0.0],
                       [0.0, 1.0, 0.0, 0.0],
                       [-sth, 0.0, cth, 0.0],
                       [0.0, 0.0, 0.0, 1.0]])
    elif self.axis == 'z':
      return np.array([[cth, -sth, 0.0, 0.0],
                      [sth, cth, 0.0, 0.0],
                      [0.0, 0.0, 1.0, 0.0],
                      [0.0, 0.0, 0.0, 1.0]])
    # Student code ends here

  def local_derivative(self):
    """ This function is used to compute the derivative of the rotation matrix.
    As before, you are supposed to make use of the self._theta parameter and the
    sin and cos functions. Similar to the function above, you will have 3 cases
    depending on the axis of rotation, which can be obtained using either
    self.axis or self.axis_index.

    Returns:
      deriv_transform: The 4x4 derivative matrix.
    """
    # Student code starts here. Implement the three derivative matrices.
    cth, sth = cos(self._theta), sin(self._theta)
    if self.axis == 'x':
      return np.array([[0.0, 0.0, 0.0, 0.0],
                       [0.0, -sth, -cth, 0.0],
                       [0.0, cth, -sth, 0.0],
                       [0.0, 0.0, 0.0, 0.0]])
    elif self.axis == 'y':
      return np.array([[-sth, 0.0, cth, 0.0],
                       [0.0, 0.0, 0.0, 0.0],
                       [-cth, 0.0, -sth, 0.0],
                       [0.0, 0.0, 0.0, 0.0]])
    elif self.axis == 'z':
      return np.array([[-sth, -cth, 0.0, 0.0],
                      [cth, -sth, 0.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0]])
    # Student code ends here

Let us test our implementation.

In [ ]:
#@title `Hinge` class unit tests

transform_list = list()
print("Test rotation transformation for x-axis with theta = 75 degrees")
hingeTest0 = Hinge("test0", axis="x", theta=(5/12 * np.pi)) # 75 degrees
print("Testing local_transform()")
ansLocalTransform0 = np.array([[ 1., 0., 0., 0.],
 [0., 0.258819, -0.965926, 0.],
 [0., 0.965926, 0.258819, 0.],
 [0., 0., 0., 1.]])
assert np.allclose(ansLocalTransform0, hingeTest0.local_transform(), atol=1e-5)
print("Passed local_transform() for test0!")
print("Testing local_derivative()")
ansLocalDerivative0 = np.array([[ 0., 0., 0., 0.],
 [0., -0.965926, -0.258819, 0.],
 [0., 0.258819, -0.965926, 0.],
 [0., 0., 0., 0.]])
assert np.allclose(ansLocalDerivative0, hingeTest0.local_derivative(), atol=1e-5)
print("Passed local_derivative() for test0!")
print("-----------------------")

print("Test rotation transformation for y-axis with theta = 45 degrees")
hingeTest1 = Hinge("test1", axis="y", theta=(1/4 * np.pi)) # 45 degrees
print("Testing local_transform()")
ansLocalTransform1 = np.array([[0.707107, 0., 0.707107, 0.],
 [0., 1., 0., 0.],
 [-0.707107, 0., 0.707107, 0.],
 [0., 0., 0., 1.]])
assert np.allclose(ansLocalTransform1, hingeTest1.local_transform(), atol=1e-5)
print("Passed local_transform() for test1!")
print("Testing local_derivative()")
ansLocalDerivative1 = np.array([[-0.707107, 0., 0.707107, 0.],
 [0., 0., 0., 0.],
 [-0.707107, 0., -0.707107, 0.],
 [0., 0., 0., 0.]])
assert np.allclose(ansLocalDerivative1, hingeTest1.local_derivative(), atol=1e-5)
print("Passed local_derivative() for test1!")
print("-----------------------")

print("Test rotation transformation for z-axis with theta = 60 degrees")
hingeTest2 = Hinge("test2", axis="z", theta=(1/3 * np.pi)) # 60 degrees
print("Testing local_transform()")
ansLocalTransform2 = np.array([[ 0.5, -0.866025, 0., 0.],
 [0.8660254, 0.5, 0., 0.],
 [0., 0., 1., 0.],
 [0., 0., 0., 1.]])
assert np.allclose(ansLocalTransform2, hingeTest2.local_transform(), atol=1e-5)
print("Passed local_transform() for test2!")
print("Testing local_derivative()")
ansLocalDerivative2 = np.array([[-0.866025, -0.5, 0., 0.],
 [0.5, -0.866025, 0., 0.],
 [0., 0., 0., 0.],
 [0., 0., 0., 0.]])
assert np.allclose(ansLocalDerivative2, hingeTest2.local_derivative(), atol=1e-5)
print("Passed local_derivative() for test2!")
print("-----------------------")

print("Passed all tests!")

# 2. Set up the skeleton.

We will now set up the structure of character, also called the skeleton.

In [ ]:
#@title `setup_character()`: Skeleton setup

# Global structure
transform_list = list()
name_to_transform = dict()
transform_list_1dof = list()
motion = list()


def setup_character():
  global transform_list, name_to_transform, transform_list_1dof
  # Reset the data structure
  transform_list = list()
  name_to_transform = dict()
  transform_list_1dof = list()
  motion = list()

  # Define the character
  base = Translation("j_tx", axis="x").add(Translation("j_ty", axis="y")).add(Translation("j_tz", axis="z"))\
    .add(Hinge("j_rx", axis="x")).add(Hinge("j_ry", axis="y")).add(Hinge("j_rz", axis="z"))\
    .add(Fixed("base"))
  base.add(Fixed("head", tz=0.5))
  for d in ['l', 'r']:
    sx = 1.0 if d == 'l' else -1.0
    base.add(Fixed("%s_base_to_shoulder" % d, tx=sx*0.2, tz=0.25))\
      .add(Hinge("j_%s_shoulder_x" % d, "x", 0.0)).add(Hinge("j_%s_shoulder_y" % d, "y", 0.0)).add(Hinge("j_%s_shoulder_z" % d, "z", 0.0))\
      .add(Fixed("%s_upperarm1" % d, tx=sx*0.15)).add(Fixed("%s_upperarm2" % d, tx=sx*0.15))\
      .add(Hinge("j_%s_elbow" % d , "z", 0.0)).add(Fixed("%s_lowerarm1" % d, tx=sx*0.15)).add(Fixed("%s_lowerarm2" % d, tx=sx*0.15))\
      .add(Fixed('%s_hand' % d, tx=sx*0.1))
  for d in ['l', 'r']:
    sx = 1.0 if d == 'l' else -1.0
    base.add(Fixed("%s_base_to_hip" % d, tx=sx*0.13, tz=-0.3))\
      .add(Hinge("j_%s_hip_x" % d, "x", 0.0)).add(Hinge("j_%s_hip_y" % d, "y", 0.0)).add(Hinge("j_%s_hip_z" % d, "z", 0.0))\
      .add(Fixed("%s_thigh1" % d, tz=-0.2)).add(Fixed("%s_thigh2" % d, tz=-0.2))\
      .add(Hinge("j_%s_knee" % d , "x", 0.0)).add(Fixed("%s_shin1" % d, tz=-0.2)).add(Fixed("%s_shin2" % d, tz=-0.2))\
      .add(Hinge("j_%s_ankle" % d , "x", 0.0)).add(Fixed("%s_feet" % d, ty=-0.12))

  # Build auxiliary data structures
  for tr in transform_list:
    name_to_transform[tr.name] = tr
    if tr.num_dofs() == 1:
      transform_list_1dof.append(tr)

  # Build dependent transformations.
  # Each joint depends on itself (if it has a DOF) as well as all its ancestors with DOFs in the skeleton hierarchy.
  for tr in transform_list:
    tr.dependent_transformations = tr.parent.dependent_transformations.copy() if tr.parent else set()

    if tr.num_dofs() == 1:
      tr.dependent_transformations.add(tr)

Let us now visualize our skeleton using PyBullet. In order to do this, we first define a base class for Rigid bodies. We then define classes for the various parts of our skeleton, namely the Box and Sphere classes, which inherit from the RigidBody class. We then use each of these components to setup the skeleton.

In [ ]:
#@title Rigidbody setup & visualization (PyBullet)

p.connect(p.DIRECT)
p.resetSimulation()

pixelWidth = 640
pixelHeight = 360
viewMatrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0,-0.3], distance=2.0, yaw=0.0, pitch=0.0, roll=0.0, upAxisIndex=2)
projectionMatrix = p.computeProjectionMatrixFOV(fov=60, aspect=pixelWidth / pixelHeight, nearVal=0.01, farVal=100)

body_list = list()
target_body_list = list()

class RigidBody(object):
  """
  A base class for rigid bodies.
  """
  def __init__(self,
      transform):
    global body_list
    self.transform = transform
    self.uid = -1
    body_list.append(self)


  def get_position_and_orientation(self):
    T = self.transform.global_transform()
    position = T[:3, 3]
    orientation = R.from_matrix(T[:3, :3]).as_quat()
    return position, orientation

  def update_pybullet(self):
    position, orientation = self.get_position_and_orientation()
    p.resetBasePositionAndOrientation(self.uid, position, orientation)


class Box(RigidBody):
  """ We use this class, which inherits from the RigidBody class, to create the
  torso and limbs of our character.
  """
  def __init__(self,
      transform,
      color = np.array([0, 1, 1, 1]),
      half_extends = [1, 1, 1],
      texUid = None):
    super().__init__(transform)

    self.visualId = p.createVisualShape(p.GEOM_BOX, halfExtents=half_extends, rgbaColor=color, specularColor=[1, 1, 1])
    position, orientation = self.get_position_and_orientation()
    self.uid = p.createMultiBody(1.0, -1, self.visualId, position, orientation)
    if texUid:
      p.changeVisualShape(self.uid, -1, textureUniqueId=texUid)

class Sphere(RigidBody):
  """ We use this class, which inherits from the RigidBody class, to create the
  hands and face of our character.
  """
  def __init__(self,
      transform,
      color = np.array([0, 1, 1, 1]),
      radius = 1.0):
    super().__init__(transform)
    self.radius = radius

    self.visualId = p.createVisualShape(p.GEOM_SPHERE, radius=radius, rgbaColor=color, specularColor=[1, 1, 1])
    position, orientation = self.get_position_and_orientation()
    self.uid = p.createMultiBody(1.0, -1, self.visualId, position, orientation)


def reset_simulation():
  global body_list
  time = 0
  p.resetSimulation()
  p.setGravity(0, 0, 0)
  p.setTimeStep(1e-6) # Dummy time step
  body_list = list()


def create_rigid_bodies():
  global name_to_transform, target_body_list
  black_rgba = [0, 0, 0, 0.64]
  white_rgba = [1, 1, 1, 0.64]
  yellow_rgba = [1, 1, 0, 0.64]
  # texUid = p.loadTexture("AK7aPaDMGbPI4WXfr9GBmMOGPiuTmOkomDPeXVKgrXzefHIiFpOQuXO8ZY1j3TG8aipbi067Q5WFqj4yn1WmS2SkJYGcQTeIww=s2560")
  # Box(name_to_transform['base'], half_extends=[0.2, 0.1, 0.3], color=white_rgba, texUid=texUid)
  Box(name_to_transform['base'], half_extends=[0.2, 0.1, 0.3], color=white_rgba)

  Sphere(name_to_transform['head'], radius=0.15, color=yellow_rgba)
  for d in ['l', 'r']:
    Box(name_to_transform['%s_upperarm1' % d], half_extends=[0.14, 0.05, 0.05], color=yellow_rgba)
    Box(name_to_transform['%s_lowerarm1' % d], half_extends=[0.14, 0.05, 0.05], color=white_rgba)
    Sphere(name_to_transform['%s_hand' % d], radius=0.05, color=yellow_rgba)
    Box(name_to_transform['%s_thigh1' % d], half_extends=[0.08, 0.06, 0.18], color=yellow_rgba)
    Box(name_to_transform['%s_shin1' % d], half_extends=[0.08, 0.06, 0.18], color=black_rgba)
    Box(name_to_transform['%s_feet' % d], half_extends=[0.05, 0.10, 0.03], color=yellow_rgba)

  max_targets = 10
  for i in range(max_targets):
    visualId = p.createVisualShape(p.GEOM_SPHERE, radius=0.06, rgbaColor=[1.0, 0.0, 0.0, 1.0], specularColor=[1, 1, 1])
    uid = p.createMultiBody(1.0, -1, visualId, [100.0, 100.0, 100.0], [0.0, 0.0, 0.0, 1.0])
    target_body_list.append(uid)

setup_character()
reset_simulation()
create_rigid_bodies()

_, _, img, _, _ = p.getCameraImage(pixelWidth, pixelHeight, viewMatrix,projectionMatrix, shadow=1, lightDirection=[1, 1, 1])
plt.imshow(img)
plt.show()

We will now define some helper functions that will allow us to connect the pose vector and the skeleton `Transformation`s.

In [ ]:
#@title Pose <-> Transformation helpers

def set_joint_positions(q):
  global transform_list_1dof
  assert len(q) == len(transform_list_1dof)
  for q_i, jnt_i in zip(q, transform_list_1dof):
    jnt_i.set_dof_value(value=q_i)


def get_joint_positions():
  global transform_list_1dof
  return np.array([jnt_i.get_dof_value() for jnt_i in transform_list_1dof])

# 3. Inverse Kinematics.
Now that we have all the required machinery in place, we will work on the final piece of the puzzle, the IK solver.


## 3.1 Jacobian function.
First, please implement the `compute_jacobian()` function according to its docstring, referring to the slides: *08_character_animation.pdf, slide 51-55*.

Specifically, given an end effector's `Transformation` (at a leaf of the skeleton hierarchy), this function should return the Jacobian of the constrained point (3D) on the end effector with respect to the skeleton's pose vector (i.e. the values of all its DOFs). So, its shape should be `(3, ndofs)`.

The key idea is to use these quantities for each joint that the end effector depends on:
1. The transformations up the kinematic chain, from the immediate parent to the root
1. The local derivative with respective to the joint's own DOF value
1. The transformations down the kinematic chain, from the joint itself to the end effector
1. Finally, the position of the constrained point in the end effector's local coordinate space ($\mathbf{h}_0$ in the slides)
    - This one is trivial in our case: *we (arbitrarily) define the constrained point to be at the origin of this local space.*

Some hints based on what the code has already computed for you so far:

- `transform_list_1dof` is a global list of transformations in the skeleton that have 1 DOF.
    - The Jacobian's columns correspond to the joints in this list only: no other joints are relevant to any Jacobian.
    - This list has been computed for you when the skeleton was built inside `setup_character()`.

- Each `Transformation` has a member variable `dependent_transformations`: a set of all `Transformation`s whose DOFs it depends on.
    - So, for a joint `j`, any joint outside the set `j.dependent_transformations` will also be irrelevant to the Jacobian of `j`.
    - This set has been computed for you for each `Transformation` in the skeleton, again inside `setup_character()`.

**<font color='orange'> \*\* Task 3: write your code below (20 pts) \*\* </font>**

**<font color='orange'> \*\* Note: The implementation of Task 3 is given. However, we recommend students to read the code carefully. \*\* </font>**

In [ ]:
def compute_jacobian(end_effector):
  """ Compute the Jacobian of the end effector. You will need to make use of the
  global_transform(), local_derivative() and transform_between() functions.
  Don't forget to consider the (trivial) case when the joint
  is not in end_effector.dependent_transformations.
  And also note that each joint has only one degree of freedom.

  Args:
    end_effector: component for which we want to find the Jacobian

  Returns:
    J: Jacobian matrix of shape (3, ndofs)
  """
  global transform_list_1dof
  ndofs = len(transform_list_1dof)
  h0 = np.array([0.0, 0.0, 0.0, 1.0]) # Our local point in a homogenous coordiate
  J = np.zeros((3, ndofs))
  for i, jnt_i in enumerate(transform_list_1dof):
    # Analytical
    if jnt_i not in end_effector.dependent_transformations:
      continue
    # Step 0. Compute T_parent. We provide this value as example.
    if jnt_i.parent:  # check if the joint has a parent (not None)
      # T_parent is the global transformation of the parent node (jnt_i.parent).
      T_parent = jnt_i.parent.global_transform()
    else:
      T_parent = np.identity(4)  # If the joint has no parent.
    # Student code starts here
    # Step 1. compute dTdq, which is the local derivative of the current joint
    dTdq = jnt_i.local_derivative()
    # Step 2. Compute T_child, which is a transformation between jnt_i and end_effector. Compute the transformation between jnt_i and end_effector.
    T_child = transform_between(jnt_i, end_effector)
    # Step 3. Compute dx_dq
    # dx_dq = (T_parent @ dTdq @ T_child)[:3, 3]
    dx_dq = T_parent @ dTdq @ T_child @ h0
    # Student code ends here
    # Because dx_dq is a 4-dim homogenous vector with the 1.0 at the end, we take the first three.
    J[:, i] = dx_dq[:3]
  return J

Let's test the Jacobian function.

In [ ]:
#@title `compute_jacobian()` unit tests

ndofs = len(transform_list_1dof)
eps = 0.0001
q = np.random.rand(ndofs) - 0.5
set_joint_positions(q)

for end_effector_name in ['l_hand', 'r_feet', 'head']:
  end_effector = name_to_transform[end_effector_name]
  J = compute_jacobian(end_effector)
  for i, jnt_i in enumerate(transform_list_1dof):
    # Analytical
    dx_dq = J[:, i]

    # Approximation
    jnt_i.set_dof_value(value=q[i] + eps)
    x_p = end_effector.global_position()
    jnt_i.set_dof_value(value=q[i] - eps)
    x_m = end_effector.global_position()
    dx_dq_approx = (x_p - x_m) / (2 * eps)
    jnt_i.set_dof_value(value=q[i])

    # # Uncomment these lines to help with debugging (they print a lot of info though)
    # print('test Jacobian for joint %s' % str(jnt_i))
    # print(dx_dq)

    assert np.allclose(dx_dq, dx_dq_approx, atol=1e-6)
  print(f'passed the test for end effector: {end_effector_name}')
print('all pass!')

## 3.2 Objective and gradient functions.
Next, please implement the objective and gradient functions, according to the respective docstrings. The gradient is with respect to the pose of the skeleton, which is now being referred to as `x` in the code instead of `q`.

**<font color='orange'> \*\* Task 4 and 5: write your code below (20 + 20 = 40 pts) \*\* </font>**

In [ ]:
end_effectors = [name_to_transform['l_hand'], name_to_transform['r_feet']]
targets = [np.array((1.2, -0.3, 0.4)), np.array((0.0, -0.2, 0.1))]


def objective_function(x):
  """ Implement the objective function here. You will need to implement
  the objective function of the form 0.5 * L2_norm(error)^2, where the
  error is the difference between each position and its target.
  Don't forget to add the contributions of all the positions/targets.

  Useful functions include global_position() and np.linalg.norm().
  You can assume that there is no additional objective (g(q) = 0).

  Args:
    x: Pose

  Returns:
    ret: The scalar value of the objective function.
  """
  global end_effectors, targets, motion
  motion.append(np.array(x))
  set_joint_positions(x) # Set the joint position: q=x.
  ret = 0.0
  for end_effector, target in zip(end_effectors, targets):
    # Student code starts here
    # Step 1. get the global position of the current end effector, h(q). You can call end_effector.global_position().
    h_q = end_effector.global_position()

    # Step 2. compute the error as |h(q) - p|^2, where p is the target.
    error = h_q - target

    # Step 3. add the error to the ret.
    ret += 0.5 * np.linalg.norm(error)**2

    # Student code ends here
  return ret

def gradient_function(x):
  """ Implement the gradient of the objective function above. Useful
  functions include global_position() and compute_jacobian().

  Args:
    x: Pose

  Returns:
    grad: The len(x)-sized gradient vector.
  """
  global end_effectors, targets
  set_joint_positions(x)
  grad = np.zeros(len(x))
  for end_effector, target in zip(end_effectors, targets):
    # Student code starts here
    # Step 1. get the position of the end effector h(q)
    h_q = end_effector.global_position()

    # Step 2. compute the jacobian J, using what you implemented (or given)
    J = compute_jacobian(end_effector)

    # Step 3. Compute the gradient as J^T C, where C = h(q) - p
    error = h_q - target

    # Step 4. Add the gradient to the grad
    grad += J.T @ error

    # Student code ends here
  return grad

We test the objective and gradient implementation.

In [ ]:
#@title Objective & Gradient unit tests

for test_id in range(10):
  x0 = -10.0 + 20.0 * np.random.rand(ndofs)
  # Exact
  g = gradient_function(x0)
  # Approximation
  g_approx = np.zeros(ndofs)
  eps = 0.001
  for i in range(ndofs):
    delta = np.zeros(ndofs)
    delta[i] = eps
    f_p = objective_function(x0 + delta)
    f_m = objective_function(x0 - delta)
    g_approx[i] = (f_p - f_m) / (eps * 2.0)
  print('test gradient against the random pose #%d' % test_id)
  assert np.allclose(g, g_approx, atol=1e-6)
print('all pass!')

Let's now solve for the motion. The targets will be added sequentially, and each time a new target is added, we will solve the pose of the character and linearly interpolate in-between frames.

In [ ]:
#@title Motion IK + interpolation (Twister) algorithm

all_end_effectors = [name_to_transform['l_hand'], name_to_transform['r_feet'], name_to_transform['r_hand'], name_to_transform['l_feet'], name_to_transform['l_upperarm2']]
all_targets = [np.array((0.6, -0.3, -0.3)), np.array((-0.2, 0.3, -1.0)), np.array((-0.5, 0.2, -0.3)), np.array((-0.5, -0.3, -0.7)), np.array((0.5, -0.3, 0.1))]
end_effectors = list()
targets = list()
motion = list()
for uid in target_body_list:
  p.resetBasePositionAndOrientation(uid, [100.0, 100.0, 100.0], [0.0, 0.0, 0.0, 1.0])
frame = 0

q0 = np.zeros(ndofs)
set_joint_positions(q0)

for idx, (end_effector, target) in enumerate(zip(all_end_effectors, all_targets)):
  p.resetBasePositionAndOrientation(target_body_list[idx], target, [0.0, 0.0, 0.0, 1.0])
  print('add constraint for: %s' % (str(end_effector)))
  # We will add a constraint one by one, and solve the corresponding pose
  end_effectors.append(end_effector)
  targets.append(target)

  # Solve the pose
  res = minimize(objective_function,
                x0=np.zeros(ndofs),
                jac=gradient_function,  # Hint: you can comment out this line to test your objective function. gradient will be computed numerically.
                method='SLSQP',
                options={'ftol': 1e-8, 'disp': True})
  print(res)

  if not os.path.exists("./frames/"):
    os.makedirs("./frames/")

  q1 = res['x']
  for w in np.linspace(0.0, 2.0, 41):
    w = min(w, 1.0)
    q = (1 - w) * q0 + w * q1
    set_joint_positions(q)
    for rb in body_list:
      rb.update_pybullet()
    # Render frame
    _, _, img, _, _ = p.getCameraImage(pixelWidth, pixelHeight, viewMatrix,projectionMatrix, shadow=1, lightDirection=[1,1,1])
    Image.fromarray(img[:, :, :3]).save('./frames/frame%04d.jpg' % frame)
    frame += 1
    if frame % 10 == 0:
      print("Rendering frame #%d" % frame)
  q0 = q1

Create a video of the motion we just solved for.

In [ ]:
#@title Motion IK + interpolation (Twister) video

ffmpeg.input('./frames/frame*.jpg', pattern_type='glob', framerate=20).output('./output.gif').overwrite_output().run()
video = io.open('./output.gif', 'r+b').read()
encoded = base64.b64encode(video)
ipythondisplay.display(HTML(data='''<img src="data:image/gif;base64,{0}"/>'''.format(encoded.decode('ascii'))))

### FK vs. IK

Can you see the animation of the character who is playing our version of the Twister game, as dictated by the targets?

You can also check the quality of IK by looking at the `fun` value of the optimization results `res`, printed by the cell containing the Twister algorithm. If it is reasonably small (< 1e-4), we can call it a success.

Imagine if you want to describe a similar motion using forward kinematics, and discuss the benefit of inverse kinematics.

**<font color='orange'> \*\*write your answer below (20 pts)\*\* </font>**


If we used forward kinematics to describe similar motions, we would need to manually find the joint rotations that would result in the body parts ending up at the desired points, and explicitly provide them to the FK position solver.

We would need to do this for every frame, and we would be responsible for making sure all the joint configurations together are physically plausible each step of the way.
In practice, this would involve many rounds of trial error, and would likely result in less realistic poses, since FK does not automatically update all joints positions.

Inverse kinematics solves precisely these issues by allowing us to specify the _end positions_ we want and solving for all joint angles and intermediate positions based on the current position constraints _automatically_.

This allows for much more intuitive, goal-oriented control, results in more natural poses, and also makes it easy to incorporate environmental constraints - all while being much less labor intensive.

## 3.3. Motion Capture (Mocap)
In this section, we visualize the motion capture data obtained from the mocap session week.


We then use inverse kinematics we implemented to retarget our robot to the human motion capture.

**There's no graded points in this section.**

### 3.3.1 Download Motion Capture Data
First, let's download the motion capture data, captured from mocap-room. Running this code will save the data in "mocap_data" folder. You can also take a look at the gifs from: https://drive.google.com/drive/folders/1IUbXcWGuujtOuAefxct24GvQwYiUE_9X





In [ ]:
#@title Download mocap data
!pip install gdown
# download motion capture files from google drive
!gdown https://drive.google.com/drive/folders/1IUbXcWGuujtOuAefxct24GvQwYiUE_9X -O mocap_data --folder --quiet

In [ ]:
#@title Load mocap data from file <br> *(un-comment your favorite motion capture data along with corresponding scale and view matrix)*

mocap_filename = "mocap_data/angela.npy"
mocap_frame_scale = 1.2/1000
viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/avery.npy"
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/david.npy"
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=90.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/actor_unknown.npy" # Forgot the name of Monday 2nd section, 2nd actor.
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=90.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/jack.npy"
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/jade.npy"
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/john.npy"
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/nicole.npy"  # one marker was detached after 6 secs
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/rahul.npy"
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/seyeon.npy"
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)

# mocap_filename = "mocap_data/william.npy"   # one marker was detached from the beginning
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=180.0, pitch=-1.0, roll=0.0, upAxisIndex=2)


# mocap_filename = "mocap_data/xinwen.npy"
# mocap_frame_scale = 1.2/1000
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=90.0, pitch=-1.0, roll=0.0, upAxisIndex=2)



mocap_frames = np.load(mocap_filename) # data in shape (n_frame, 6, 4) array, where 6 means number of end effector, 4 is (x,y,z, occluded_flag)
mocap_frame_rep = ['_Spine', '_RightFoot', '_RightHand', '_LeftFoot', '_Head', '_LeftHand']  # order of end_effector inside data

Our data is recorded in 120Hz, meaning each frame is 8.3 miliseconds apart.

We reduce the size of our data by taking only 2 second of entire frame starting from 600th frame. We also reduce the framerate to 30Hz by taking only every four frames.

In [ ]:
#@title Frame pre-processing

# First, make all occluded frame become previous non-occluded frame
for f_idx in range(mocap_frames.shape[0]):
  mocap_frame = mocap_frames[f_idx]
  for ee_idx in range(mocap_frame.shape[0]):
    ee_occluded = mocap_frame[ee_idx, 3]
    if ee_occluded and f_idx > 0:
      mocap_frames[f_idx, ee_idx, :3] = mocap_frames[f_idx-1, ee_idx, :3]


start_frame_idx = 0
end_frame_idx = 1200
total_render_frames = (end_frame_idx - start_frame_idx) // 4

# Our data is recorded 120Hz, meaning each frame is 8.3 milisecond apart. We only take 1/4 of the data, making it 30Hz
target_mocap_frames = mocap_frames[start_frame_idx:end_frame_idx:4, :, :3]
print(target_mocap_frames.shape)

# normalize x,y axis
target_mocap_frames_mean = np.mean(target_mocap_frames, axis=(0,1))
target_mocap_frames_mean[2] = 0

# mocap_frame_scale = 1.2/1000  # scale to match the size of our object.
# mocap_frame_scale = 1.1/1000  # scale to match the size of our object.

target_mocap_frames = (target_mocap_frames - target_mocap_frames_mean) * mocap_frame_scale

Visualize our Mocap Data to check it is valid

In [ ]:
#@title Mocap visualization & validation

from matplotlib import animation
# Do 3D plot of data
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

ax.set(xlim3d=(-1,1), xlabel='X')
ax.set(ylim3d=(-1, 1), ylabel='Y')
ax.set(zlim3d=(-0.5, 1.5), zlabel='Z')
ax.view_init(10, 70)

rods = [_] * 10
rods[0], = ax.plot([], [], 'ro-') # spline -> Head
rods[1], = ax.plot([], [], 'ko-') # spline -> Right Foot
rods[2], = ax.plot([], [], 'ko-') # spline -> Right Hand
rods[3], = ax.plot([], [], 'ko-') # spline -> Left Foot
rods[4], = ax.plot([], [], 'ko-') # spline -> Left Hand

idx_spline = 0
idx_rightfoot = 1
idx_righthand = 2
idx_leftfoot = 3
idx_head = 4
idx_lefthand = 5

x=0
y=1
z=2

def plot_scene(f_idx):
  frame = target_mocap_frames[f_idx]

  rods[0].set_data([frame[idx_spline, x], frame[idx_head,x]], [frame[idx_spline, y], frame[idx_head, y]])
  # NOTE: there is no .set_data() for 3 dim data...
  rods[0].set_3d_properties([frame[idx_spline, z], frame[idx_head,z]])

  rods[1].set_data([frame[idx_spline, x], frame[idx_rightfoot,x]], [frame[idx_spline, y], frame[idx_rightfoot, y]])
  rods[1].set_3d_properties([frame[idx_spline, z], frame[idx_rightfoot,z]])

  rods[2].set_data([frame[idx_spline, x], frame[idx_righthand,x]], [frame[idx_spline, y], frame[idx_righthand, y]])
  rods[2].set_3d_properties([frame[idx_spline, z], frame[idx_righthand,z]])

  rods[3].set_data([frame[idx_spline, x], frame[idx_leftfoot,x]], [frame[idx_spline, y], frame[idx_leftfoot, y]])
  rods[3].set_3d_properties([frame[idx_spline, z], frame[idx_leftfoot,z]])

  rods[4].set_data([frame[idx_spline, x], frame[idx_lefthand,x]], [frame[idx_spline, y], frame[idx_lefthand, y]])
  rods[4].set_3d_properties([frame[idx_spline, z], frame[idx_lefthand,z]])

# Called before the first frame
def init_animation():
  plot_scene(0)
  return []


# Called every frame
def animate(i):
  if i % 10 == 0:
      print("Generating frame #%d..." % i)
  plot_scene(i)
  return []

anim = animation.FuncAnimation(fig, animate, init_func=init_animation, frames=total_render_frames, interval=33, blit=False) # 30Hz
# HTML(anim.to_html5_video())
HTML(anim.to_jshtml())

### 3.3.2 Retarget our Motion Capture to our Figure using Inverse Kinematics


In [ ]:
#@title Retargeting mocap to PyBullet via IK: Algorithm

# Add plane
import pybullet_data as pd
p.setAdditionalSearchPath(pd.getDataPath())
planeId = p.loadURDF("plane.urdf")

# You can use different viewMatrix depending on your mocap data for better visualization
# Rotate Yaw 180 degrees to make "left" to be +x direction as in our mocap-sytstem. You can modify this line for better view of the character
# viewMatrix_mocap = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0, 0.8], distance=3.5, yaw=90.0, pitch=-1.0, roll=0.0, upAxisIndex=2)


all_end_effectors = [name_to_transform['r_feet'], name_to_transform['r_hand'], name_to_transform['l_feet'], name_to_transform['head'], name_to_transform['l_hand']]

end_effectors = list()
targets = list()
motion = list()
for uid in target_body_list:
  p.resetBasePositionAndOrientation(uid, [100.0, 100.0, 100.0], [0.0, 0.0, 0.0, 1.0])
frame = 0

q0 = np.zeros(ndofs)
set_joint_positions(q0)

q = q0
duration_list = []
# for idx in range(60): # don't print all frames..
for idx in range(300): # don't print all frames..
  frame = target_mocap_frames[idx]


  end_effectors = all_end_effectors
  targets = [frame[idx_rightfoot, :3], frame[idx_righthand, :3], frame[idx_leftfoot, :3], frame[idx_head, :3], frame[idx_lefthand, :3]]
  for t_idx, target in enumerate(targets):
    p.resetBasePositionAndOrientation(target_body_list[t_idx], target, [0.0, 0.0, 0.0, 1.0])

  # Solve the pose
  res = minimize(objective_function,
                x0=q, # Start from previous pose
                jac=gradient_function,  # Hint: you can comment out this line to test your objective function. gradient will be computed numerically.
                method='SLSQP',
                options={'ftol': 1e-4, 'disp': False})  # increase ftol for faster rendering
                # options={'ftol': 1e-8, 'disp': False})
  # print(res)
  q = res['x']
  set_joint_positions(q)
  for rb in body_list:
    rb.update_pybullet()
  # Render frame
  _, _, img, _, _ = p.getCameraImage(pixelWidth, pixelHeight, viewMatrix_mocap,projectionMatrix, shadow=1, lightDirection=[1,1,1])
  Image.fromarray(img[:, :, :3]).save('./frames/frame_mocap%04d.jpg' % idx)
  if idx % 10 == 0:
    print("Rendering frame #%d" % idx)

In [ ]:
#@title Retargeting mocap to PyBullet via IK: Video

ffmpeg.input('./frames/frame_mocap*.jpg', pattern_type='glob', framerate=33).output('./output_mocap.gif').overwrite_output().run()
video = io.open('./output_mocap.gif', 'r+b').read()
encoded = base64.b64encode(video)
ipythondisplay.display(HTML(data='''<img src="data:image/gif;base64,{0}"/>'''.format(encoded.decode('ascii'))))

# 4. Extra Questions

## 4.1 Discussing Motion Capture Result (New?)

For some motion capture data, the retargeted motion doesn't resemble the actual human actor. Discuss what are the causes of it and what are some ways to make the motion of our robot to resemble that of human actor.


**<font color='orange'> \*\*write your answer below (2 pts)\*\* </font>**

Our inverse kinematics solver enforces reasonably realistic joint angles throughout the animation by disallowing dramatic violations of the basic degrees of freedom of the joints in the human body.
However, to further ensure the retargeted motion resembles real human movements, we would need to implement additional custom constraints into our solver system.

These additional constraints could take the form of explicit heuristics to nudge the joint angles and positions towards more natural-looking movements, prevent physically impossible orientations, or enforce extra physical constraints such as energy minimization.
We could also use data-driven machine learning approaches to align the movements with recorded human movements.

In addition to the algorithmic limitations, there are also limitations to the motion capture technology and setting we used.
For example, only five points on the body are tracked, requiring e.g. the knee and elbow positions to be inferred algorithmically.
There are also limitations to the spatiotemporal granularity of the captured positions, as well as potential errors in estimating the true 3D positions of the emitters from the joint 2D projections recorded by cameras.

Finally, we made no effort to incorporate hard constraints from the physical environment, such as ensuring the ground cannot be penetrated.

Here are some approaches we could take to better align the motion with that of a human:
- Add more (passive) markers and/or (active) emitters to the actor during motion capture.
- Add more cameras at different positions/angles relative to the performer.
- Incorporate other recorded optical data to more accurately infer the poses using machine-learning/data-driven methods.
- Incorporate additional heuristic, environment, and/or physics-informed constraints into the IK solver.

## 4.2. Memoization of `transform_between()`

The `transform_between()` function is one of the busiest functions in the code. First, count the number of times it is called during one evaluation of the objective function.

By caching the computation results, you may be able to save some function calls, which is called "*memoization*" (https://en.wikipedia.org/wiki/Memoization). Implement this caching mechanism and see if you can reduce the number of `transform_between()` function calls per objective evaluation.

**Note:** *don't forget to erase the cache when we change the pose.*

**<font color='orange'> \*\*edit the above code directly and explain your implementation & results - number of function calls - below (4 pts)\*\* </font>**

(explain your answer here)

## 4.3. Never Leave The Targets

At the final stage of the Twister video (after adding the final constraint), some of the character's end-effectors may leave the targets temporarily during the interpolation of pose. Refer to the frame below as an example *(if you don't see something like it in the Twister video for some reason, please give it harder target configurations)*.

This is because we first solve the constrained IK problem once per stage (as each target is added), and then interpolate the motion freely in the pose space between the poses for each stage. Nothing forces the poses generated *during the interpolation* to satisfy all constraints that the targets specify.

Please update the motion interpolation algorithm to prevent this behavior.

**Hint:** *you may need to interpolate in the space of the targets instead, and solve the resulting IK problems more often.*

**<font color='orange'> \*\*edit the above code directly and explain your implementation below (4 pts)\*\* </font>**

![Bad Frame](http://cc.gatech.edu/~sha9/public/P5-twister-bad.png)

(explain your answer here).